In [2]:
from google.colab import files
uploaded = files.upload()



Saving train.csv to train.csv


In [79]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import os
from sentence_transformers import SentenceTransformer, util
import gradio as gr
import torch
from pyngrok import ngrok
import threading





In [129]:
print(os.listdir())


['.config', 'train.csv', 'unnamed (3).jpg', 'train (1).csv', '.gradio', 'unnamed (2).jpg', 'unnamed.jpg', 'context_embeddings.npy', 'train (2).csv', 'unnamed (1).jpg', 'sample_data']


In [84]:
df = pd.read_csv("train.csv")
df["Context"] = df["Context"].fillna("")
df["Response"] = df["Response"].fillna("")

model = SentenceTransformer('multi-qa-mpnet-base-dot-v1')

contexts = df["Context"].tolist()
responses = df["Response"].tolist()
context_embeddings = model.encode(contexts, convert_to_tensor=True)

SIMILARITY_THRESHOLD = 0.55
TOP_K = 5

def chatbot(user_input):
    user_input = user_input.strip()
    if not user_input:
        return "You can write something so we can start chatting."

    user_embedding = model.encode(user_input, convert_to_tensor=True)
    cos_scores = util.cos_sim(user_embedding, context_embeddings)[0]
    top_k_scores, top_k_indices = torch.topk(cos_scores, k=TOP_K)
    best_score = top_k_scores[0].item()
    best_idx = top_k_indices[0].item()
    if best_score >= SIMILARITY_THRESHOLD:
        return responses[best_idx]
    else:
        return "Could you explain what you mean more?"

css = """
.gradio-container {
    background: url("https://images.unsplash.com/photo-1507525428034-b723cf961d3e") no-repeat center center fixed;
    background-size: cover;
    font-family: Arial, sans-serif;
}
.chat-title {
    color: white;
    font-size: 36px;
    font-weight: bold;
    text-align: center;
    margin-bottom: 20px;
    text-shadow: 2px 2px 4px rgba(128,128,128,0.8);
}
.message-container {
    display: flex;
    flex-direction: column;
    gap: 15px;
    margin: 10px 0;
}
.user-message, .bot-message {
    max-width: 70%;
    padding: 10px 15px;
    border-radius: 20px;
    word-wrap: break-word;
}
.user-message {
    background-color: #ffffff;
    align-self: flex-end;
    margin-bottom: 10px;
    box-shadow: 0 2px 5px rgba(0,0,0,0.1);
}
.bot-message {
    background-color: #e5e5e5;
    align-self: flex-start;
    margin-bottom: 10px;
    box-shadow: 0 2px 5px rgba(0,0,0,0.1);
}
input[type="text"] {
    border-radius: 20px !important;
    padding: 10px;
    width: 100% !important;
}
"""

with gr.Blocks(css=css) as iface:
    gr.HTML("<div class='chat-title'>Mental Health Chatbot</div>")
    chat_history = gr.HTML("<div class='message-container'></div>")
    user_input = gr.Textbox(label="Type your message here", placeholder="Type here...")

    def respond(user_text, history):
        bot_reply = chatbot(user_text)
        new_history = history + f"""
        <div class='user-message'>{user_text}</div>
        <div class='bot-message'>{bot_reply}</div>
        """
        return new_history, ""

    user_input.submit(respond, [user_input, chat_history], [chat_history, user_input])

iface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d948c0758f7611f9b0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
